In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from loaders import *
import yaml
from yaml import load

In [ ]:
def fitness(dataflow):
    '''
    Calculates the fitness of a given permutation `dataflow`. Returns the
    inverse EDP after running the timeloop mapper on this dataflow.
    '''
    data = self.evaluate(dataflow)
    energy, latency = data
    inverse_EDP = 1 / (energy * latency)
    print(f"{dataflow} has a fitness of {inverse_EDP}")

    # update visited
    self.VISITED[''.join(dataflow)] = inverse_EDP
    self.mapper_calls.append([''.join(dataflow), inverse_EDP])
    return inverse_EDP


def evaluate(dataflow):
    '''
    Evaluates the given dataflow on -- architecture

    dataflow: computation ordering in list format
    workload: the file path to the workload this is being evaluated on
    returns tuple of energy, latency
    '''

    # create a new constraints file with the new PE permutation
    stream = open(self.constraints, 'r')
    dictionary = yaml.safe_load(stream)
    # print(dictionary['constraints']['targets'][4])
    idx = 4 # PE
    dictionary['constraints']['targets'][idx]['permutation'] = dataflow

    filename = ''.join(dataflow)
    with open(f'iters/configs/{filename}.yaml', 'w') as file:
        yaml.dump(dictionary, file, default_flow_style=False)

    constraints = f'iters/configs/{filename}.yaml'

    sys_1x16_result = run_timeloop_mapper( # TODO: this should be run_timeloop_mapper not run_timeloop_model!
        # config,
        self.pe_dims,
        architecture=self.architecture,
        mapper=self.mapper,
        problem=self.workload,
        constraints=constraints 
    )

    # keeping track of mapper call count for data purposes
    self.mapper_call_count += 1
    
    stats = open('./output_dir/timeloop-mapper.stats.txt', 'r').read()
    mapping = sys_1x16_result.mapping

    lines = stats.split('\n')
    energy = float([l for l in lines if 'Energy:' in l][0].split(' ', 2)[1])
    cycles = int([l for l in lines if 'Cycles:' in l][0].split(' ', 1)[1])
    # min_energy = min(min_energy, energy)

    print(energy, cycles)
    return energy, cycles

In [ ]:
dataflow = ['R', 'S', 'P', 'Q', 'C', 'M', 'N']

stream = open('conv2_2x8', 'r')
visited = yaml.safe_load(stream)

population = [random.sample(self.dataflow, len(dataflow)) for _ in range(121)]